In [84]:
import pandas as pd

In [85]:
df = pd.read_csv('../data/lesson_3_data.csv', encoding='utf-8', sep=',')

In [86]:
df.head(3)

,tc,cta,id_art,id_subsubfam,id_subfam,id_famn,id_seccion,id_subagr,id_agr,vta,uni,id_artn,art_sp,fam_sp,fam_en
0,613111122,54691661,17119,101070640100,1010706401,10107064,101,107,101,13.56,5,17119,FIDEOS FINOS 250 G Brand_3,PASTA ALIMENTICIA SE,pasta
1,526910959,62375978,17826,101070640100,1010706401,10107064,101,107,101,11.75,2,17826,MARAVILLA 250 G Brand_2,PASTA ALIMENTICIA SE,pasta
2,574963196,73946190,77291,101070640100,1010706401,10107064,101,107,101,6.63,5,77291,TALLARINES 1 KG Brand_3,PASTA ALIMENTICIA SE,pasta


In [87]:
user_df = df[['tc', 'art_sp']]

In [88]:
user_df = user_df.rename(columns={'tc': 'user_id', 'art_sp': 'brand_info'})

In [89]:
user_df['brand_name'] = user_df['brand_info'].apply(lambda brand_info: brand_info.split(' ')[-1])

In [90]:
user_df.head(3)

,user_id,brand_info,brand_name
0,613111122,FIDEOS FINOS 250 G Brand_3,Brand_3
1,526910959,MARAVILLA 250 G Brand_2,Brand_2
2,574963196,TALLARINES 1 KG Brand_3,Brand_3


In [91]:
user_df.shape

(30000, 3)

In [98]:
users_purchases = user_df.groupby('user_id', as_index=False) \
    .agg({'brand_name': 'count'}) \
    .rename(columns={'brand_name': 'purchases'}) \
    .query('purchases >= 9')

In [120]:
users_unique_brands = user_df.groupby('user_id', as_index=False) \
    .agg({'brand_name': pd.Series.nunique}) \
    .rename(columns={'brand_name': 'unique_brands'})

Не все пользователи для анализа нам нужны, потому лояльность заключается в том, что пользователь неоднократно проявляет активность к товарам определенного бренда. А если пользователь совершил всего 1 покупку, то какой смысл его рассматривать?

In [106]:
users_purchases.shape

(735, 2)

In [105]:
users_purchases.head(3)

,user_id,purchases
3,100329503,9
7,100789544,9
8,101447723,11


In [107]:
users_purchases.describe()

,user_id,purchases
count,7.350000e+02,735.000000
mean,5.315716e+08,10.110204
std,2.637867e+08,1.321626
min,1.003295e+08,9.000000
25%,2.938252e+08,9.000000
50%,5.270166e+08,10.000000
75%,7.608032e+08,11.000000
max,9.999718e+08,17.000000


In [116]:
users_by_lovely_brand_df = user_df.groupby(['user_id', 'brand_name'], as_index=False) \
    .agg({'brand_info': 'count'}) \
    .rename(columns={'brand_info': 'purchases'}) \
    .sort_values(['user_id', 'purchases'], ascending=[False, False]) \
    .groupby('user_id') \
    .head(1) \
    .rename(columns={'brand_name': 'lovely_brand_name', 'purchases': 'lovely_brand_purchases'})